In [2]:
import numpy as np
import pandas as pd
from hmmlearn import hmm



In [4]:
# Step 1: Data Preprocessing
data = pd.read_csv('../data/GDELT_THA.csv')
print(len(data))
data.dropna(inplace=True)
print(len(data))
# Convert string dates to datetime objects
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d')

# Step: Encode Categorical Variables

# Extract date components
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

data.drop(columns=['Date'], inplace=True)

data['Source'] = data['Source'].astype('category')
data['Target'] = data['Target'].astype('category')

print(data.tail())



731154
696577
        Source  Target  CAMEOCode  NumEvents  NumArts  QuadClass  Goldstein  \
731149  USACVL     THA         40          1       20          1        1.0   
731150  USAMED     THA         70          1        2          2        7.0   
731151  USAMED  THAMED         70          1        2          2        7.0   
731152     USA     THA         61          3       15          2        6.4   
731153     VNM     THA        160          1        2          4       -4.0   

        SourceGeoType  SourceGeoLat  SourceGeoLong  TargetGeoType  \
731149            1.0       38.0000       -97.0000            4.0   
731150            4.0       32.9049        35.8072            4.0   
731151            4.0       13.7500       100.5170            4.0   
731152            1.0       38.0000       -97.0000            1.0   
731153            4.0       10.7500       106.6670            1.0   

        TargetGeoLat  TargetGeoLong  ActionGeoType  ActionGeoLat  \
731149       25.0000        

In [ ]:
# Example how to initialize and train the HMM
model = hmm.GaussianHMM(n_components=2)
X = data[['Latitude', 'Longitude', 'Time']].values
model.fit(X)

# Step: Inference